<h1 style="text-align:center;">K-PROTOTYPES++</h1>

## Importando librerías

In [45]:
import pandas as pd
import numpy as np

## Cargando el archivo

In [2]:
# archivo = '15datos.csv'
archivo = 'depresion_estudiantes.csv'
# archivo = '4.csv'
tiene_indices = 'si'.lower()

In [3]:
df = pd.read_csv(archivo, sep=',')
if tiene_indices == 'si'.lower():
    df = df.iloc[:, 1:]
df.head()

,Género,Edad,Presión Académica,Satisfacción de estudio,Horas de Estudio
0,Masculino,28,4,5,7
1,Masculino,23,1,4,7
2,Femenino,31,1,5,4
3,Masculino,19,4,4,1
4,Femenino,34,4,2,6


## Implementando el algoritmo

In [5]:
class Estadistica:
    @classmethod
    def distancia_euclidiana(cls, arrays, centro):
        distancias_list = []
        distancia = 0
        for arr in arrays:
            for i in range(len(arr)):
                distancia += (arr[i] - centro[i])**2
            distancias_list.append(round(np.sqrt(distancia), 2))
            distancia = 0
        return np.array(distancias_list)

    @classmethod
    def similitud(cls, arrays, centro):
        similitud_list = []
        similitud = 0
        for arr in arrays:
            for i in range(len(arr)):
                if arr[i] != centro[i]:
                    similitud += 1    
            similitud_list.append(similitud)
            similitud = 0
        return np.array(similitud_list)

    @classmethod
    def minimos(cls, *arrays):
        minimos = []
        for i in range(len(arrays[0])):
            valores = [arr[i] for arr in arrays]
            min_valor = min(valores)
            min_index = valores.index(min_valor)
            min_columna = f'C{min_index + 1}'
            minimos.append([min_valor, min_columna, i])
        return minimos

    @classmethod
    def promedios(cls, dataframe):
        columnas = dataframe.columns
        nuevo_centroide = []
        for columna in columnas:
            if dataframe[columna].dtype == 'object' or dataframe[columna].dtype == 'string':
                if len(dataframe[columna].mode()) > 1:
                    # Aquí se puede escoger la moda dependiendo de cada uno
                    nuevo_centroide.append(dataframe[columna].mode()[0])
                else:
                    nuevo_centroide.append(dataframe[columna].mode()[0])
                    
            else:
                nuevo_centroide.append(round(dataframe[columna].mean(), 2))
        return np.array(nuevo_centroide)

    @classmethod
    def normalizar_numericos(cls, df):
        # Separando datos categóricos y numéricos
        numericos_df = df.select_dtypes(['float64', 'int64'])
        categoricos_df = df.select_dtypes(['object','string'])
        # Normalizando los datos numéricos
        numericos_df = round((numericos_df - numericos_df.min()) / (numericos_df.max() - numericos_df.min()), 2)
        return numericos_df, categoricos_df

    @classmethod
    def is_numeric(cls, value):
        try:
            float(value)
            return True
        except ValueError:
            return False

In [36]:
class KPrototypesPlus:
    def __centroides_probabilidad(self, cent, df, peso, primera_pos):
        # Separando datos categóricos y numéricos (numéricos ya normalizados)
        numericos_df, categoricos_df = Estadistica.normalizar_numericos(df)
    
        # Generando los centroides (obteniendo el mejor centride de acuerdo a probabilidades)
        indice_max = -1
        indices = [primera_pos]
        distancias_finales = []
        for i in range(cent):
            # Localizando centroides
            centroide_numerico = numericos_df.iloc[primera_pos] if i == 0 else numericos_df.iloc[indice_max]
            centroide_categorico = categoricos_df.iloc[primera_pos] if i == 0 else categoricos_df.iloc[indice_max]
            
            # Calculando las distancias euclidianas y similitud de coeficientes
            d_e = Estadistica.distancia_euclidiana(numericos_df.to_numpy(), centroide_numerico.to_numpy())
            s = Estadistica.similitud(categoricos_df.to_numpy(), centroide_categorico.to_numpy())
            
            # Calulando las distancias finales
            dist_final = d_e + s * 1
            distancias_finales.append(dist_final)
        
            if not (len(distancias_finales) > 1):
                # Calculando el indice del nuevo centroide
                probabiliad = dist_final / sum(dist_final)
                max_cent = max(probabiliad)
                indice_max = np.where(probabiliad == max_cent)[0][0]
                indices.append(indice_max)
            else:
                # Obteniendo los minimos de las distancias
                m = Estadistica.minimos(*distancias_finales)
                m = np.array(np.array(m)[:, 0], dtype='float64')
                
                # Calculando el indice del nuevo centroide
                probabiliad = m / sum(m)
                max_cent = max(probabiliad)
                indice_max = np.where(probabiliad == max_cent)[0][0]
                indices.append(indice_max)
        return indices

    
    def _k_prototypes(self, k, df, peso, *args, inicio=1, fin=10, pos_fin=None):
        # Separando datos categóricos y numéricos (numéricos ya normalizados)
        numericos_df, categoricos_df = Estadistica.normalizar_numericos(df)
    
        # Generando los centroides
        centroides_numericos = []
        centroides_categoricos = []
        distancias_e = []
        similitud_coef = []
    
        if pos_fin is None:
            for i in range(k):
                # Numéricos
                centroides_numericos.append(numericos_df.iloc[args[i]])
                # Categóricos
                centroides_categoricos.append(categoricos_df.iloc[args[i]])
        
            # Calculando las distancias euclidianas y similitud de coeficientes
            for i in range(k):
                distancias_e.append(Estadistica.distancia_euclidiana(numericos_df.to_numpy(), centroides_numericos[i].to_numpy()))
                similitud_coef.append(Estadistica.similitud(categoricos_df.to_numpy(), centroides_categoricos[i].to_numpy()))
        else:
            fila_n = []
            fila_c = []
            for row in pos_fin:
                for dato in row:
                    if Estadistica.is_numeric(dato):    
                        fila_n.append(dato)
                    else:
                        fila_c.append(dato)
                centroides_numericos.append(fila_n) # Numéricos
                centroides_categoricos.append(fila_c) # Categóricos
                fila_n = []
                fila_c = []
    
            centroides_numericos = np.array(centroides_numericos, dtype='float64')
            centroides_categoricos = np.array(centroides_categoricos, dtype='object')
            # Calculando las distancias euclidianas y similitud de coeficientes
            for i in range(k):
                distancias_e.append(Estadistica.distancia_euclidiana(numericos_df.to_numpy(), centroides_numericos[i]))
                similitud_coef.append(Estadistica.similitud(categoricos_df.to_numpy(), centroides_categoricos[i]))
           
    
        # Calulando las distancias finales
        distancias_cent = []
        for i in range(k):
            dist_final = distancias_e[i] + similitud_coef[i] * peso
            distancias_cent.append(dist_final)
    
        # Obteniendo los minimos y sus etiquetas. Después conviertiendolo a un dataframe
        m = Estadistica.minimos(*distancias_cent)
        salida = pd.DataFrame(m, columns=['Minimo', 'Etiqueta', 'Indice'])
    
        # Calculando los promedios de cada centroide con su etiqueta
        etiquetas = [f'C{i+1}' for i in range(k)]
        nuevos = []
        titulo = f' Iteración {inicio} '
        print(titulo.center(70, '-'))
        for i in range(k):
            indice = salida[salida['Etiqueta'] == etiquetas[i]]['Indice']
            nuevo_centriode = pd.concat([categoricos_df.iloc[indice], numericos_df.iloc[indice]], axis=1)
            # print('\n', nuevo_centriode)
            prom = Estadistica.promedios(nuevo_centriode)
            nuevos.append(prom)
            print(f'\nNuevo {etiquetas[i]} = {prom}')
        print()
    
        # Validando recursividad
        if np.array_equal(nuevos, pos_fin) or inicio == fin:
            return nuevos, etiquetas, salida, pd.concat([categoricos_df, numericos_df], axis=1)
        else:
            return self._k_prototypes(k, df, peso, *args, inicio=inicio+1, pos_fin=nuevos)


    
    def k_prototypes_plus(self, df, k, peso, semilla):
        try:
            # Establecer la semilla
            np.random.seed(semilla)
            
            # Generar un número aleatorio entero dentro del rango de los datos
            numero_entero = np.random.randint(0, len(df))
            # print(numero_entero)
            # Calculando el k-prototypes++
            centroides = self.__centroides_probabilidad(cent=k-1, df=df, peso=peso, primera_pos=numero_entero) #  Obteniendo los centroides
            print(' K-PROTOTYPES++ '.center(100, '_'), '\n')
            print('\t\t\t\tCentroides en las posiciones:', centroides, '\n')
            return self._k_prototypes(k, df, peso, *centroides)
        except Exception as e:
            print(f'Ocurrio un error: {e}')
            return -1

In [38]:
k_prototypes_plus = KPrototypesPlus()
centroide_final = k_prototypes_plus.k_prototypes_plus(df=df, k=3, peso=1, semilla=13)

__________________________________________ K-PROTOTYPES++ __________________________________________ 

				Centroides en las posiciones: [2, 10, 9] 

---------------------------- Iteración 1 -----------------------------

Nuevo C1 = ['Femenino' '0.7' '0.5' '0.67' '0.5']

Nuevo C2 = ['Masculino' '0.21' '0.8' '0.5' '0.4']

Nuevo C3 = ['Masculino' '0.48' '0.25' '0.31' '0.82']

---------------------------- Iteración 2 -----------------------------

Nuevo C1 = ['Femenino' '0.7' '0.5' '0.67' '0.5']

Nuevo C2 = ['Masculino' '0.21' '0.8' '0.5' '0.4']

Nuevo C3 = ['Masculino' '0.48' '0.25' '0.31' '0.82']



## Exportando datos finales

In [44]:
# centroide_final

In [39]:
df_etiquetas = centroide_final[-1] # Recupeperando los datos noramalizados
df_etiquetas['Etiqueta'] = centroide_final[2]['Etiqueta'] # salida (concatenando sus etiquetas)

archivo_excel = 'archivo_final.xlsx'
df_etiquetas.to_excel(archivo_excel, index=False) # Exportando a un archivo

In [40]:
# Uniendo las etiquetas de los centroies finales y colocandolo en un DataFrame
centroides, etiquetas= centroide_final[0], centroide_final[1]
promedio_centroides = []
for i, _ in enumerate(centroides):
    centroide = centroides[i].tolist()
    centroide.append(etiquetas[i])
    promedio_centroides.append(centroide)
promedio_centroides = pd.DataFrame(promedio_centroides, columns=df_etiquetas.columns)

In [41]:
promedio_centroides

,Género,Edad,Presión Académica,Satisfacción de estudio,Horas de Estudio,Etiqueta
0,Femenino,0.7,0.5,0.67,0.5,C1
1,Masculino,0.21,0.8,0.5,0.4,C2
2,Masculino,0.48,0.25,0.31,0.82,C3


In [42]:
# Si el archivo ya existe, anexamos los nuevos datos
with pd.ExcelWriter(archivo_excel, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    # promedio_centroides.to_excel(writer, index=False, sheet_name='Sheet1', startrow=writer.sheets['Sheet1'].max_row, header=False) # fila
    promedio_centroides.to_excel(writer, index=False, sheet_name='Sheet1', startcol=writer.sheets['Sheet1'].max_column + 1, header=True) # columna